# Zadanie 3: Rozmieszczenie liczb zmiennoprzecinkowych

## Część 3.1: Badanie przedziału [1, 2]

### 1. Opis problemu
Zadanie polega na eksperymentalnym sprawdzeniu, czy w arytmetyce `Float64` (standard IEEE 754) liczby zmiennoprzecinkowe w przedziale $[1, 2]$ są równomiernie rozmieszczone. Mamy zweryfikować, czy krok (odstęp) między kolejnymi liczbami jest stały i wynosi $\delta = 2^{-52}$. Sprawdzimy to, obliczając odstęp między $1.0$ a następną liczbą maszynową, a także analizując reprezentację binarną (używając `bitstring`) kolejnych liczb w tym zakresie.

In [ ]:
using Printf

# --- Definicje dla Float64 ---
# 1 bit znaku (S), 11 bitów wykładnika (E), 52 bity mantysy (F)
# S EEEEEEEEEEE FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF

# Funkcja pomocnicza do formatowania bitstring dla Float64
function format_bitstring(f64::Float64)
    s = bitstring(f64)
    # Dzieli bitstring na znak(1), wykładnik(11) i mantysę(52)
    return s[1:1] * " " * s[2:12] * " " * s[13:end]
end

println("--- Badanie przedziału [1.0, 2.0] ---")

# Definiujemy delta = 2^-52
delta_teor = 2.0^(-52)
@printf "Teoretyczna delta (2^-52): \t%.17e\n" delta_teor
@printf "Wartość eps(1.0): \t\t%.17e\n" eps(1.0)

# Bierzemy 1.0 jako punkt startowy
x1 = 1.0
println("\nLiczba x1 = 1.0")
println("Bitstring x1: \t", format_bitstring(x1))

# Znajdujemy następną liczbę maszynową na dwa sposoby:
x2_nextfloat = nextfloat(x1)
x2_teoria = 1.0 + delta_teor

println("\nLiczba x2 (następna po 1.0)")
println("Bitstring x2 (z nextfloat): \t", format_bitstring(x2_nextfloat))
println("Bitstring x2 (z 1.0 + delta): \t", format_bitstring(x2_teoria))
@printf "Wartość x2 (z nextfloat): \t%.17e\n" x2_nextfloat

println("\nCzy (1.0 + 2^-52) == nextfloat(1.0)? \t", x2_teoria == x2_nextfloat)

# Sprawdzamy kolejną liczbę (k=2)
x3_nextfloat = nextfloat(x2_nextfloat)
x3_teoria = 1.0 + 2*delta_teor
println("\nLiczba x3 (k=2)")
println("Bitstring x3 (z nextfloat): \t", format_bitstring(x3_nextfloat))
println("Bitstring x3 (z 1.0 + 2*delta): \t", format_bitstring(x3_teoria))
println("Czy (1.0 + 2*2^-52) == nextfloat(nextfloat(1.0))? \t", x3_teoria == x3_nextfloat)

--- Badanie przedziału [1.0, 2.0] ---
Teoretyczna delta (2^-52): 	2.22044604925031308e-16
Wartość eps(1.0): 		2.22044604925031308e-16

Liczba x1 = 1.0
Bitstring x1: 	0 01111111111 0000000000000000000000000000000000000000000000000000

Liczba x2 (następna po 1.0)
Bitstring x2 (z nextfloat): 	0 01111111111 0000000000000000000000000000000000000000000000000001
Bitstring x2 (z 1.0 + delta): 	0 01111111111 0000000000000000000000000000000000000000000000000001
Wartość x2 (z nextfloat): 	1.00000000000000022e+00

Czy (1.0 + 2^-52) == nextfloat(1.0)? 	true

Liczba x3 (k=2)
Bitstring x3 (z nextfloat): 	0 01111111111 0000000000000000000000000000000000000000000000000010
Bitstring x3 (z 1.0 + 2*delta): 	0 01111111111 0000000000000000000000000000000000000000000000000010
Czy (1.0 + 2*2^-52) == nextfloat(nextfloat(1.0))? 	true


### 3. Wyniki i interpretacja
**Interpretacja:**
Wyniki w pełni potwierdzają hipotezę.
1.  **Zgodność $\delta$:** Teoretyczna wartość $\delta = 2^{-52}$ jest identyczna z wartością `eps(1.0)`. Obie wynoszą `2.22044604925031308e-16`.
2.  **Krok `k=1`:**
    * `x1 = 1.0` ma wykładnik `01111111111` (reprezentujący $2^0$) i mantysę `...000`.
    * `x2 = nextfloat(1.0)` oraz `x2 = 1.0 + delta` dały identyczne wyniki.
    * Analiza `bitstring` dla `x2` pokazuje ten sam wykładnik $2^0$, a mantysa zmieniła się na `...001`.
3.  **Krok `k=2`:**
    * Obliczenie `1.0 + 2*delta` dało identyczny wynik co dwukrotne wywołanie `nextfloat(1.0)`.
    * `bitstring` dla `x3` pokazuje mantysę `...010`.

Wszystkie liczby w przedziale $[1, 2)$ mają ten sam wykładnik $E=0$ (zapisany jako `01111111111` z biasem 1023). Wartość liczby to $1.F \times 2^0$, gdzie $F$ to 52 bity mantysy. Zmiana ostatniego bitu mantysy (bitu o wadze $2^{-52}$) przesuwa nas do kolejnej liczby maszynowej.

### 4. Wnioski
Eksperyment potwierdził, że w **standardzie IEEE 754** dla `Float64`, liczby w przedziale $[1, 2]$ są rozmieszczone **równomiernie (liniowo)**.
* Odstęp między nimi (nazywany ULP - Unit in the Last Place) jest stały i wynosi $\delta = 2^{-52}$, co jest równe `eps(1.0)` (czyli $macheps$).
* Każda liczba $x$ w tym przedziale może być zapisana jako $x = 1.0 + k \cdot 2^{-52}$, gdzie $k$ jest liczbą całkowitą od $0$ do $2^{52}$ (co obejmuje $1.0$ i $2.0$).
* W **obliczeniach maszynowych** oznacza to, że błąd bezwzględny (absolutny) w tym przedziale jest stały, ale błąd względny maleje wraz ze wzrostem $x$ od 1 do 2.

---

## Część 3.2: Badanie przedziałów [0.5, 1] oraz [2, 4]

### 1. Opis problemu
Należy teraz zbadać, jak rozmieszczone są liczby zmiennoprzecinkowe `Float64` w sąsiednich przedziałach potęg dwójki: $[0.5, 1]$ oraz $[2, 4]$. Naszym celem jest znalezienie kroku $\delta$ dla każdego z tych przedziałów i określenie ogólnej formy reprezentacji liczb.

In [3]:
using Printf

# Używamy tej samej funkcji pomocniczej
function format_bitstring(f64::Float64)
    s = bitstring(f64)
    return s[1:1] * " " * s[2:12] * " " * s[13:end]
end

println("--- Badanie przedziału [2.0, 4.0] ---")
y1 = 2.0
y2_nextfloat = nextfloat(y1)
delta_y = y2_nextfloat - y1

println("Liczba y1 = 2.0")
println("Bitstring y1: \t", format_bitstring(y1))
println("\nLiczba y2 (następna po 2.0)")
println("Bitstring y2: \t", format_bitstring(y2_nextfloat))

@printf "\nObliczony krok delta_y = y2-y1: \t%.17e\n" delta_y
@printf "Wartość eps(2.0): \t\t%.17e\n" eps(2.0)
@printf "Czy delta_y == 2^-51? \t\t%s\n" delta_y == 2.0^(-51)


println("\n--- Badanie przedziału [0.5, 1.0] ---")
z1 = 0.5
z2_nextfloat = nextfloat(z1)
delta_z = z2_nextfloat - z1

println("Liczba z1 = 0.5")
println("Bitstring z1: \t", format_bitstring(z1))
println("\nLiczba z2 (następna po 0.5)")
println("Bitstring z2: \t", format_bitstring(z2_nextfloat))

@printf "\nObliczony krok delta_z = z2-z1: \t%.17e\n" delta_z
@printf "Wartość eps(0.5): \t\t%.17e\n" eps(0.5)
@printf "Czy delta_z == 2^-53? \t\t%s\n" delta_z == 2.0^(-53)

--- Badanie przedziału [2.0, 4.0] ---
Liczba y1 = 2.0
Bitstring y1: 	0 10000000000 0000000000000000000000000000000000000000000000000000

Liczba y2 (następna po 2.0)
Bitstring y2: 	0 10000000000 0000000000000000000000000000000000000000000000000001

Obliczony krok delta_y = y2-y1: 	4.44089209850062616e-16
Wartość eps(2.0): 		4.44089209850062616e-16
Czy delta_y == 2^-51? 		true

--- Badanie przedziału [0.5, 1.0] ---
Liczba z1 = 0.5
Bitstring z1: 	0 01111111110 0000000000000000000000000000000000000000000000000000

Liczba z2 (następna po 0.5)
Bitstring z2: 	0 01111111110 0000000000000000000000000000000000000000000000000001

Obliczony krok delta_z = z2-z1: 	1.11022302462515654e-16
Wartość eps(0.5): 		1.11022302462515654e-16
Czy delta_z == 2^-53? 		true


### 3. Wyniki i interpretacja
**Przedział [2.0, 4.0]:**
1.  `y1 = 2.0`: `bitstring` `0 10000000000 ...000` pokazuje wykładnik $2^1$.
2.  `y2 = nextfloat(2.0)`: `bitstring` `0 10000000000 ...001` pokazuje ten sam wykładnik $2^1$, a mantysa zmieniła się na `...001`.
3.  Obliczony krok $\delta_y = y2 - y1$ wynosi `4.44089209850062616e-16`.
4.  Ta wartość jest **identyczna** z `eps(2.0)` oraz z wartością teoretyczną $2^{-51}$.

**Przedział [0.5, 1.0]:**
1.  `z1 = 0.5`: `bitstring` `0 01111111110 ...000` pokazuje wykładnik $2^{-1}$.
2.  `z2 = nextfloat(0.5)`: `bitstring` `0 01111111110 ...001` pokazuje ten sam wykładnik $2^{-1}$, a mantysa zmieniła się na `...001`.
3.  Obliczony krok $\delta_z = z2 - z1$ wynosi `1.11022302462515654e-16`.
4.  Ta wartość jest **identyczna** z `eps(0.5)` oraz z wartością teoretyczną $2^{-53}$.

### 4. Wnioski
Eksperyment pokazał, że liczby zmiennoprzecinkowe **nie są** rozmieszczone równomiernie w całym zakresie liczb rzeczywistych, ale są równomiernie rozmieszczone wewnątrz przedziałów $[2^E, 2^{E+1})$.

* W przedziale $[2, 4]$ (gdzie $E=1$), krok $\delta_y = 2^{-51}$, czyli $2 \times \delta_{poprzedni}$. Liczby mają postać $x = 2.0 + k \cdot 2^{-51}$.
* W przedziale $[0.5, 1]$ (gdzie $E=-1$), krok $\delta_z = 2^{-53}$, czyli $\frac{1}{2} \times \delta_{poprzedni}$. Liczby mają postać $x = 0.5 + k \cdot 2^{-53}$.

W **standardzie IEEE 754** odstęp (ULP) między liczbami podwaja się przy każdym przekroczeniu potęgi dwójki. Wartość $\text{ulp}(x)$ (lub `eps(x)` w Julii) jest skalowana przez wykładnik liczby. W **obliczeniach maszynowych** oznacza to, że choć **błąd bezwzględny** rośnie wraz z wielkością liczby, **błąd względny** (czyli $\text{ulp}(x) / x$) pozostaje w przybliżeniu stały (w granicach $macheps$).